In [16]:
from pydub import AudioSegment
import time
import datetime
import os
import numpy as np

import pickle

import random
from xml.dom import minidom
from librosa import load
import shutil

from extract_feat import extract_feats_single_wav
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

from mlxtend.feature_selection import SequentialFeatureSelector as sfs

path_caregiver = '..//speaker_id_module//SpeakerID//singles//1-caregiver//'
dest_caregiver = '..//app//2-Training//singles//1-caregiver//'

path_patient = '..//speaker_id_module//SpeakerID//singles//2-patient//'
dest_patient = '..//app//2-Training//singles//2-patient//'

def slice_audios(path, dest):
    for old_audio in os.listdir(dest):
        os.remove(dest + old_audio)
        
    for audio_index in range(0, len(os.listdir(path))):
        target_audio_path =  path + os.listdir(path)[audio_index]
        print('input audio: ' + target_audio_path)

        target_audio = AudioSegment.from_wav(target_audio_path)
        target_duration = target_audio.duration_seconds

        folds = int(target_duration/5.0)
        for fold in range(0, folds+1):

            start_time = time.time()

            start_time = fold * 5000  # Works in milliseconds
            end_time = (fold + 1) * 5000
            new_audio = target_audio[start_time:end_time]

            components = target_audio_path.split('/')
            filename = components[len(components)-1]

            new_audio_path = dest + filename[:len(filename)-4] + '_' + str(fold) + '.wav'
            print('generated the slice of the audio segment at index ' + str(fold) )
            new_audio.export(new_audio_path, format="wav")

def change_amplitude(emotionfile, d1, newSoundFile, d2):
    
    if d1 <= d2:
        sound = AudioSegment.from_file(emotionfile) - np.random.randint(0, (6 * d2/d1 - 1))
        sound.export(newSoundFile, format='wav')  ### save the new generated file in a folder
    else:
        print('Invalid distance parameters. d1 should be <= d2.')

def change_amplitude_range(emotionfile, newSoundFile, threshold):
    #amount = np.random.randint(0, threshold)
    amount = random.uniform(0, threshold)
    #print('Deamplify ' + str(emotionfile) + ' by ' + str(amount) + ' db.')
    sound = AudioSegment.from_file(emotionfile) - amount
    sound.export(newSoundFile, format='wav')  ### save the new generated file in a folder
    return amount

def deamplify_per_folder(directory):
    for file in os.listdir(directory):
        if file.endswith('.wav'):
            soundFile = directory + file
            newSoundFile = directory + 'deamplified_' + file
            change_amplitude_range(soundFile, newSoundFile, 12)

def add_noise_and_deamplify_per_folder(directory, extension, noise_directory):
    for file in os.listdir(directory):
        if file.endswith(extension) and not file[1] == '_':
            for i in range(0, 2):
                soundFile = directory + file
                amount = change_amplitude_range(soundFile, soundFile, 6)
                noise = random.choice(os.listdir(noise_directory))
                random_noise = noise_directory + noise
                newSoundFile = directory + 'deamp_' + str(amount) + '_noise_' + noise[:len(noise)-5] + '_' + file
                add_noise_per_file(soundFile, random_noise, newSoundFile)
                print(newSoundFile)

def add_noise_per_file(emotionfile, bgnoise, newSoundFile):
    
    emotionsound = AudioSegment.from_wav(emotionfile)
    emotion_duration = emotionsound.duration_seconds * 1000
    noise = AudioSegment.from_wav(bgnoise)
    noise_duration = noise.duration_seconds * 1000
    
    threshold = noise_duration - emotion_duration
    
    if threshold > 0:
        overlay_start = np.random.randint(0, threshold)
    else:
        overlay_start = 0
        
    targeted_chunk = noise[overlay_start:overlay_start + emotion_duration]
    newSound = emotionsound.overlay(targeted_chunk, position=0)
    newSound=newSound[0:5000]
    newSound.export(newSoundFile, format='wav')  ### save the new generated file in a folder

def extract_features_for_all_wavs(dest, label):
    result = np.expand_dims(np.zeros((48, 272)), axis=0)

    for wav in os.listdir(dest):
        vec = extract_feats_single_wav(dest + wav)
        if not str(vec.shape) == '(48, 272)':
            continue
        result = np.vstack((result, np.expand_dims(vec, axis=0)))

    result = result[1:]
    labels = np.expand_dims(np.asarray([label] * len(result)), axis=1)
    print(result.shape)
    print(labels.shape)

    return result, labels


In [17]:
import tensorflow as tf

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.constraints import maxnorm
from keras.layers import Convolution1D, Dense, MaxPooling1D, Flatten, Add, Dropout, Input, Activation
from keras.layers import TimeDistributed, Bidirectional, LSTM, LeakyReLU
from keras.models import Sequential
from keras import optimizers, regularizers
from keras.utils import np_utils, to_categorical
from keras.models import Model, load_model, Sequential
from keras.regularizers import l2

import keras

from IPython.display import clear_output
from tensorflow.python.client import device_lib
from tensorflow.python.keras import backend
import tensorflow as tf

In [18]:
# STEP 1: slice into 5-second wavs
slice_audios(path_caregiver, dest_caregiver)
slice_audios(path_patient, dest_patient)
noise_directory = '..//noise_home//'

add_noise_and_deamplify_per_folder(dest_caregiver, '.wav', noise_directory)
add_noise_and_deamplify_per_folder(dest_patient, '.wav', noise_directory)


input audio: ..//speaker_id_module//SpeakerID//singles//1-caregiver//caregiver.wav
generated the slice of the audio segment at index 0
generated the slice of the audio segment at index 1
generated the slice of the audio segment at index 2
generated the slice of the audio segment at index 3
generated the slice of the audio segment at index 4
generated the slice of the audio segment at index 5
generated the slice of the audio segment at index 6
generated the slice of the audio segment at index 7
generated the slice of the audio segment at index 8
generated the slice of the audio segment at index 9
generated the slice of the audio segment at index 10
generated the slice of the audio segment at index 11
generated the slice of the audio segment at index 12
generated the slice of the audio segment at index 13
generated the slice of the audio segment at index 14
generated the slice of the audio segment at index 15
generated the slice of the audio segment at index 16
generated the slice of the

..//app//2-Training//singles//1-caregiver//deamp_1.332488361492251_noise_home_82_caregiver_9.wav
..//app//2-Training//singles//1-caregiver//deamp_3.1097178981844067_noise_home_27_caregiver_9.wav
..//app//2-Training//singles//1-caregiver//deamp_3.6564562257168705_noise_home_25_caregiver_10.wav
..//app//2-Training//singles//1-caregiver//deamp_3.7447357689257585_noise_home_82_caregiver_10.wav
..//app//2-Training//singles//1-caregiver//deamp_5.08082838970032_noise_home_88_caregiver_11.wav
..//app//2-Training//singles//1-caregiver//deamp_4.739877500838018_noise_home_80_caregiver_11.wav
..//app//2-Training//singles//1-caregiver//deamp_3.3445120771739125_noise_home_2_caregiver_12.wav
..//app//2-Training//singles//1-caregiver//deamp_3.4822921672781906_noise_home_25_caregiver_12.wav
..//app//2-Training//singles//1-caregiver//deamp_4.839261365345314_noise_home_2_caregiver_13.wav
..//app//2-Training//singles//1-caregiver//deamp_5.685788503236472_noise_home_80_caregiver_13.wav
..//app//2-Training/

..//app//2-Training//singles//1-caregiver//deamp_3.3465629757480695_noise_home_22_caregiver_51.wav
..//app//2-Training//singles//1-caregiver//deamp_2.751932477690522_noise_home_26_caregiver_51.wav
..//app//2-Training//singles//1-caregiver//deamp_5.73075295742193_noise_home_83_caregiver_52.wav
..//app//2-Training//singles//1-caregiver//deamp_0.19233157091554265_noise_home_88_caregiver_52.wav
..//app//2-Training//singles//1-caregiver//deamp_1.5597850430539297_noise_home_82_caregiver_53.wav
..//app//2-Training//singles//1-caregiver//deamp_4.518751400704388_noise_home_81_caregiver_53.wav
..//app//2-Training//singles//1-caregiver//deamp_2.9439320122442756_noise_home_2_caregiver_54.wav
..//app//2-Training//singles//1-caregiver//deamp_1.7776300720920013_noise_home_22_caregiver_54.wav
..//app//2-Training//singles//1-caregiver//deamp_1.2105199963245301_noise_home_23_caregiver_55.wav
..//app//2-Training//singles//1-caregiver//deamp_5.40777879025104_noise_home_25_caregiver_55.wav
..//app//2-Train

..//app//2-Training//singles//2-patient//deamp_3.65034075503405_noise_home_24_patient_34.wav
..//app//2-Training//singles//2-patient//deamp_3.6169933784152715_noise_home_82_patient_35.wav
..//app//2-Training//singles//2-patient//deamp_2.610435311852764_noise_home_23_patient_35.wav
..//app//2-Training//singles//2-patient//deamp_1.026687551826642_noise_home_2_patient_36.wav
..//app//2-Training//singles//2-patient//deamp_4.369336146946861_noise_home_24_patient_36.wav
..//app//2-Training//singles//2-patient//deamp_1.638042457987909_noise_home_88_patient_37.wav
..//app//2-Training//singles//2-patient//deamp_1.6942384180274572_noise_home_80_patient_37.wav
..//app//2-Training//singles//2-patient//deamp_1.2615519062212788_noise_home_27_patient_38.wav
..//app//2-Training//singles//2-patient//deamp_2.618063788974895_noise_home_23_patient_38.wav
..//app//2-Training//singles//2-patient//deamp_2.4224083277845194_noise_home_83_patient_39.wav
..//app//2-Training//singles//2-patient//deamp_0.899872361

In [19]:
# Step 2: get the vecs of shape (X, 48, 272)
X_caregiver, y_caregiver = extract_features_for_all_wavs(dest_caregiver, 0)
X_patient, y_patient = extract_features_for_all_wavs(dest_patient, 1)

X = np.vstack((X_caregiver, X_patient))
y = to_categorical( np.vstack((y_caregiver, y_patient)) )

#nsamples, nx, ny = X.shape
#X = X.reshape((nsamples,nx*ny))

X, X_test, y, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=42)

(177, 48, 272)
(177, 1)
(180, 48, 272)
(180, 1)


In [20]:
import tensorflow as tf

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.constraints import maxnorm
from keras.layers import Convolution1D, Dense, MaxPooling1D, Flatten, Add, Dropout, Input, Activation
from keras.layers import TimeDistributed, Bidirectional, LSTM, LeakyReLU
from keras.models import Sequential
from keras import optimizers, regularizers
from keras.utils import np_utils, to_categorical
from keras.models import Model, load_model, Sequential
from keras.regularizers import l2

import keras

from IPython.display import clear_output
from tensorflow.python.client import device_lib
from tensorflow.python.keras import backend
import tensorflow as tf


print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.keras.backend.clear_session()
tf.compat.v1.reset_default_graph()

def mil_squared_error(y_true, y_pred):
    return tf.keras.backend.square(tf.keras.backend.max(y_pred) - tf.keras.backend.max(y_true))

adam = tf.keras.optimizers.Adam(learning_rate=1e-5)

def train_cnn():

    model = keras.Sequential()
    model.add(Convolution1D(filters= 1500, kernel_size=2, strides=2, activation='relu', input_shape=X_train[0].shape))
    model.add(MaxPooling1D(2))
    model.add(Dropout(0.2))

    #model = keras.Sequential()
    model.add(Convolution1D(filters= 500, kernel_size=2, strides=2, activation='relu'))
    model.add(MaxPooling1D(2))
    model.add(Dropout(0.2))

    '''
    model = keras.Sequential()
    model.add(Convolution1D(filters= 500, kernel_size=1, strides=1, activation='relu'))
    model.add(MaxPooling1D(2))
    model.add(Dropout(0.2))

    model = keras.Sequential()
    model.add(Convolution1D(filters= 500, kernel_size=2, strides=2, activation='relu'))
    model.add(MaxPooling1D(2))
    model.add(Dropout(0.2))
    '''
    
    model.add(Flatten())

    for i in range(0, 1):
        model.add(Dense(256, activation="relu"))
        model.add(Dropout(0.2))
    model.add(Dense(2, activation="softmax"))
    model.compile(loss=['categorical_crossentropy'], optimizer=adam, metrics=['accuracy', mil_squared_error])
    
    print("Fit model on training data")
    history = model.fit(
        X_train,
        y_train,
        batch_size=64,
        epochs=150,
        validation_data=(X_val, y_val), verbose=1
    )

    #from sklearn.metrics import f1_score

    y_preds = [np.argmax(val) for val in model.predict(X_test)]
    y_trues = [np.argmax(val) for val in y_test]
    print(accuracy_score(y_trues, y_preds))

    model.save('..//models//cnn.hdf5')

    return model


Num GPUs Available:  0


In [21]:
model = train_cnn()
try:
    del intermediate_layer_model
except:
    pass

intermediate_layer_model = keras.Model(inputs=model.input,
                                    outputs=model.get_layer(index=len(model.layers)-2).output)
print('intermediate model summary: ')
intermediate_layer_model.summary()
intermediate_layer_model.save('..//models//intermediate_layer_model_cnn.hdf5')

print('full model summary: ')
model.summary()

Fit model on training data
Train on 179 samples, validate on 60 samples
Epoch 1/150
179/179 [==============================] - 1s 5ms/step - loss: 0.7027 - accuracy: 0.4749 - mil_squared_error: 0.1673 - val_loss: 0.6965 - val_accuracy: 0.4667 - val_mil_squared_error: 0.1993
Epoch 2/150
179/179 [==============================] - 1s 4ms/step - loss: 0.6993 - accuracy: 0.4749 - mil_squared_error: 0.1670 - val_loss: 0.6937 - val_accuracy: 0.4500 - val_mil_squared_error: 0.2050
Epoch 3/150
179/179 [==============================] - 1s 4ms/step - loss: 0.6944 - accuracy: 0.4972 - mil_squared_error: 0.1575 - val_loss: 0.6906 - val_accuracy: 0.4667 - val_mil_squared_error: 0.2090
Epoch 4/150
179/179 [==============================] - 1s 4ms/step - loss: 0.6771 - accuracy: 0.5866 - mil_squared_error: 0.1649 - val_loss: 0.6877 - val_accuracy: 0.5000 - val_mil_squared_error: 0.2114
Epoch 5/150
179/179 [==============================] - 1s 3ms/step - loss: 0.6739 - accuracy: 0.5978 - mil_squared_e

Epoch 41/150
179/179 [==============================] - 1s 4ms/step - loss: 0.4989 - accuracy: 0.9553 - mil_squared_error: 0.0600 - val_loss: 0.5400 - val_accuracy: 0.9000 - val_mil_squared_error: 0.0858
Epoch 42/150
179/179 [==============================] - 1s 4ms/step - loss: 0.5024 - accuracy: 0.9274 - mil_squared_error: 0.0700 - val_loss: 0.5347 - val_accuracy: 0.9167 - val_mil_squared_error: 0.0814
Epoch 43/150
179/179 [==============================] - 1s 3ms/step - loss: 0.4813 - accuracy: 0.9665 - mil_squared_error: 0.0647 - val_loss: 0.5292 - val_accuracy: 0.9167 - val_mil_squared_error: 0.0769
Epoch 44/150
179/179 [==============================] - 1s 4ms/step - loss: 0.4863 - accuracy: 0.9665 - mil_squared_error: 0.0593 - val_loss: 0.5239 - val_accuracy: 0.9167 - val_mil_squared_error: 0.0736
Epoch 45/150
179/179 [==============================] - 1s 3ms/step - loss: 0.4593 - accuracy: 0.9777 - mil_squared_error: 0.0484 - val_loss: 0.5184 - val_accuracy: 0.9167 - val_mil_sq

179/179 [==============================] - 1s 4ms/step - loss: 0.2479 - accuracy: 0.9888 - mil_squared_error: 0.0035 - val_loss: 0.3190 - val_accuracy: 0.9500 - val_mil_squared_error: 0.0067
Epoch 82/150
179/179 [==============================] - 1s 4ms/step - loss: 0.2445 - accuracy: 0.9944 - mil_squared_error: 0.0033 - val_loss: 0.3145 - val_accuracy: 0.9333 - val_mil_squared_error: 0.0057
Epoch 83/150
179/179 [==============================] - 1s 3ms/step - loss: 0.2240 - accuracy: 0.9944 - mil_squared_error: 0.0024 - val_loss: 0.3095 - val_accuracy: 0.9333 - val_mil_squared_error: 0.0054
Epoch 84/150
179/179 [==============================] - 1s 3ms/step - loss: 0.2312 - accuracy: 0.9888 - mil_squared_error: 0.0021 - val_loss: 0.3044 - val_accuracy: 0.9500 - val_mil_squared_error: 0.0053
Epoch 85/150
179/179 [==============================] - 1s 4ms/step - loss: 0.2292 - accuracy: 0.9721 - mil_squared_error: 0.0032 - val_loss: 0.2999 - val_accuracy: 0.9333 - val_mil_squared_error: 

179/179 [==============================] - 1s 4ms/step - loss: 0.1124 - accuracy: 0.9888 - mil_squared_error: 2.4038e-04 - val_loss: 0.1788 - val_accuracy: 0.9667 - val_mil_squared_error: 3.7049e-04
Epoch 121/150
179/179 [==============================] - 1s 3ms/step - loss: 0.1080 - accuracy: 0.9944 - mil_squared_error: 1.3659e-04 - val_loss: 0.1757 - val_accuracy: 0.9667 - val_mil_squared_error: 3.6880e-04
Epoch 122/150
179/179 [==============================] - 1s 3ms/step - loss: 0.1059 - accuracy: 0.9944 - mil_squared_error: 1.7521e-04 - val_loss: 0.1732 - val_accuracy: 0.9667 - val_mil_squared_error: 3.5219e-04
Epoch 123/150
179/179 [==============================] - 1s 4ms/step - loss: 0.1037 - accuracy: 0.9888 - mil_squared_error: 2.3633e-04 - val_loss: 0.1712 - val_accuracy: 0.9667 - val_mil_squared_error: 3.1829e-04
Epoch 124/150
179/179 [==============================] - 1s 3ms/step - loss: 0.1058 - accuracy: 0.9944 - mil_squared_error: 1.5575e-04 - val_loss: 0.1691 - val_ac

In [22]:
def get_emp_miu(X, y):
    outputs = intermediate_layer_model.predict(X)
    norms = [np.linalg.norm(output) for output in outputs]
    emp_miu = np.mean(norms)
    
    path = '..//models//emp_miu_class_' + str(y) + '.npy'
    print('emp_miu_class_' + str(y) + ' = ' + str(emp_miu))
    np.save(path, emp_miu)
    return emp_miu

emp_miu_caregiver = get_emp_miu(X_caregiver, 0)
emp_miu_patient = get_emp_miu(X_patient, 1)

emp_miu_class_0 = 4.362856
emp_miu_class_1 = 4.6469145


In [23]:
def get_emp_sigma(emp_miu_0, emp_miu_1, X_0, X_1):

    X_0 = intermediate_layer_model.predict(X_0)
    X_1 = intermediate_layer_model.predict(X_1)

    X_0_norms = [np.linalg.norm(x) for x in X_0]
    X_1_norms = [np.linalg.norm(x) for x in X_1]

    class_0 = [ (x-emp_miu_0) * (x-emp_miu_0) for x in X_0_norms] # should be transpose if not just a num
    class_1 = [ (x-emp_miu_1) * (x-emp_miu_1) for x in X_1_norms]
    emp_sigma = np.sum(class_0 + class_1)/(len(class_0) + len(class_1))
    
    '''
    class_0 = [ (x-emp_miu_0) @ np.transpose(x-emp_miu_0) for x in intermediate_layer_model.predict(X_0) ]
    class_1 = [ (x-emp_miu_1) @ np.transpose(x-emp_miu_1) for x in intermediate_layer_model.predict(X_1) ]

    emp_sigma = (class_0 + class_1)/(len(class_0) + len(class_1))
    '''

    print('the emprical covar matrix is ' + str(emp_sigma.shape))

    path = '..//models//inv_emp_sigma.npy'

    try:
        result = np.linalg.pinv(emp_sigma)
    except:
        result = emp_sigma

    np.save(path, result)
    return emp_sigma

emp_sigma = get_emp_sigma(emp_miu_caregiver, emp_miu_patient, X_caregiver, X_patient)

the emprical covar matrix is ()


In [24]:
def get_emp_mahalanobis(X, y):
    mahalanobis_coeff = 0

    emp_miu = np.load('..//models//emp_miu_class_' + str(y) + '.npy')
    inv_emp_sigma = np.load('..//models//inv_emp_sigma.npy')

    try:
        mahalanobis_dists = [np.linalg.norm(np.transpose(x-emp_miu) @ inv_emp_sigma @ (x-emp_miu)) for x in intermediate_layer_model.predict(X)]
    except:
        mahalanobis_dists = [np.linalg.norm((x-emp_miu) * inv_emp_sigma * (x-emp_miu)) for x in intermediate_layer_model.predict(X)]

    mahalanobis_mean = np.mean(mahalanobis_dists)
    mahalanobis_std = np.std(mahalanobis_dists)

    print('mahalanobis mean for class ' + str(y) + ' is ' + str(mahalanobis_mean))
    print('mahalanobis std for class ' + str(y) + ' is ' + str(mahalanobis_std))

    np.save('..//models//mahalanobis_mean_class_' + str(y) + '.npy', mahalanobis_mean)
    np.save('..//models//mahalanobis_std_class_' + str(y) + '.npy', mahalanobis_std)
    
    # np.linspace(0, 200, 2000, endpoint=False)
    for coeff in np.linspace(0, 100, 100000, endpoint=False):
        upper = mahalanobis_mean + coeff*mahalanobis_std
        lower = mahalanobis_mean - coeff*mahalanobis_std
        
        valid_xs = []
        for x in mahalanobis_dists:            
            if lower < x and x < upper:
                valid_xs.append(x)

        if len(valid_xs)/len(X) > 0.75:
            print(len(valid_xs)/len(X))
            mahalanobis_coeff = coeff
            np.save('..//models//mahalanobis_threshold_coefficient_class_' + str(y) + '.npy', coeff)
            print('the mahalanobis threshold coefficient for class ' + str(y) + ' is ' + str(coeff))
            break

    return mahalanobis_mean, mahalanobis_std, mahalanobis_coeff


#emp_sigma = get_emp_sigma(emp_miu_caregiver, emp_miu_patient, X_caregiver, X_patient)

In [25]:
m_mean_0, m_std_0, m_coeff_0 = get_emp_mahalanobis(X_caregiver, 0)
m_mean_1, m_std_1, m_coeff_1 = get_emp_mahalanobis(X_patient, 1)

mahalanobis mean for class 0 is 24.455559
mahalanobis std for class 0 is 0.076576635
0.751412429378531
the mahalanobis threshold coefficient for class 0 is 1.172
mahalanobis mean for class 1 is 27.875002
mahalanobis std for class 1 is 0.063289374
0.7555555555555555
the mahalanobis threshold coefficient for class 1 is 1.1380000000000001


In [11]:
def load_emp_miu(y):
    path = 'models//emp_miu_class_' + str(y) + '.npy'
    return np.load(path)

def load_inv_emp_covar():
    path = 'models//inv_emp_sigma.npy'
    return np.load(path)

def load_mahalanobis_mean(y):
    path = 'models//mahalanobis_mean_class_' + str(y) + '.npy'
    return np.load(path)

def load_mahalanobis_std(y):
    path = 'models//mahalanobis_std_class_' + str(y) + '.npy'
    return np.load(path)

def load_mahalanobis_coeff(y):
    path = 'models//mahalanobis_threshold_class_' + str(y) + '.npy'
    return np.load(path)

def detect_ood(x, predicted_y):
    
    assert(predicted_y == '0' or predicted_y == '1')
       
    emp_miu = load_emp_miu(y)
    inv_emp_sigma = load_inv_emp_covar()
    m_mean = load_mahalanobis_mean(y)
    m_std = load_mahalanobis_std(y)
    coeff = load_mahalanobis_coeff(y)
    
    upper = m_mean + coeff*m_std
    lower = m_mean - coeff*m_std

    try:
        m_dist = np.transpose(x-emp_miu) @ inv_emp_sigma @ (x-emp_miu)
    except:
        m = (x-emp_miu) * inv_emp_sigma * (x-emp_miu)
    
    if lower < m and m < upper:
        return True
    else:
        return False